Ideas for how to implement simple two-layer climate model with interactive components
========

Useful info:
* https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
* (possibly) https://stackoverflow.com/questions/31911884/interactive-plots-placement-in-ipython-notebook-widget

TODO: 
* Better control layout - nice to get some sort of grid.
* Fit model against obs using minimization.
* Get updated AR6 forcing data.
* Get SSP scenario data for future.
* Dropdown for SSP scenario.
* Simple instructions on how to get working.
* Why is uncertainty lacking one year (runs 1850-2015).
* Rationalize all input data CSV files.
* RMSE vs obs. (Values seem different to Excel)

DONE: 
* **Check/compare model output with Excel.** Was using lambda=0.08 instead of 0.8.
* **Get obs data as CSV and compare against obs.** Done, plus uncertainty.
* **Check it works using AppsAnywhere on windows.** Works for me with no extra setup.

Updates:
* 29/7/22: Initial playing. Import forcing data (exported from Excel as CSV). Implement simple model which is a copy of the Excel CLIMATE MODEL tab. Get interactive controls working with checkboxes and sliders.
* 29/7/22: Try on AppsAnywhere. Fix problem with lambda value. Get obs data.

In [1]:
from climate_model import ClimateModel
cm = ClimateModel()
cm.display_controls()

interactive(children=(Dropdown(description='Future scenaro:', options=(None, 'SSP1-1.9', 'SSP1-2.6', 'SSP2-4.5…